# SLU08 - Classification With Logistic Regression: Exercise notebook

In [30]:
import pandas as pd 
import numpy as np 
import hashlib

In this notebook you will practice the following: 

    - What classification is for
    - Logistic regression
    - Cost function
    - Binary classification
    
You thought that you would get away without implementing your own little Logistic Regression? Hah!


# Exercise 1. Implement the Exponential part of Sigmoid Function


In the first exercise you will implement **only the piece** of the sigmoid function where you have to use an exponential. 

Here's a quick reminder of the formula:

$$\hat{p} = \frac{1}{1 + e^{-z}}$$

In this exercise we only want you to complete the exponential part given the values of b0, b1, x1, b2 and x2:

$$e^{-z}$$

Recall that z has the following formula:

$$z = \beta_0 + \beta_1 x_1 + \beta_2 x_2$$

**Hint: Divide your z into pieces by Betas, I've left the placeholders in there!**

In [31]:
def exponential_z(b0, b1, x1, b2, x2):
    """ 
    Implementation of the exponential part of 
    the sigmoid function manually. In this exercise you 
    have to compute the e raised to the power -z. Z is calculated
    according to the following formula: b0+b1x1+b2x2. 
    
    You can use the inputs given to generate the z.
    
    Args:
        b0 (np.float64): value of the intercept
        b1 (np.float64): value of first coefficient 
        x1 (np.float64): value of first variable
        b2 (np.float64): value of second coefficient 
        x2 (np.float64): value of second variable

    Returns:
        exp_z (np.float64): the exponential part of
        the sigmoid function

    """
    
    # hint: obtain the exponential part
    # using np.exp()
    
    # YOUR CODE HERE
    exp_z = np.exp(-(b0+b1*x1+b2*x2))
    #raise NotImplementedError()
    return exp_z

In [32]:
value_arr = [1, 1, 2, 2, 0.4]

exponential= exponential_z(
    value_arr[0], value_arr[1], value_arr[2], value_arr[3], value_arr[4])

np.testing.assert_almost_equal(np.round(exponential,3), 0.022)

Expected output:

    Exponential part: 0.02

# Exercise 2: Make a Prediction

The next step is to implement a function that receives an observation and returns the predicted probability with the sigmoid function.

For instance, we can make a prediction given a model with data and coefficients by using the sigmoid:

$$\hat{p} = \frac{1}{1 + e^{-z}}$$

Where Z is the linear equation - you can't use the same function that you used above for the Z part as the input are now two arrays, one with the train data (x1, x2, ..., xn) and another with the coefficients (b0, b1, .., bn).

**Complete here:**

In [33]:
def predict_proba(train, coefficients):
    """ 
    Implementation of a function that returns 
    predicted probabilities for an observation.
    
    In the train array you will have 
    the data values (corresponding to the x1, x2, .. , xn).
    
    In the coefficients array you will have
    the coefficients values (corresponding to the b0, b1, .., bn).
    
    In this exercise you should be able to return a float 
    with the calculated probabilities given an array of size (1, n). 
    The resulting value should be a float (the predicted probability)
    with a value between 0 and 1. 
    
    Note: Be mindful that the input is completely different from 
    the function above - you receive two arrays in this functions while 
    in the function above you received 5 floats - each corresponding
    to the x's and b's.
    
    Args:
        train (np.array): a numpy array of shape (n)
            - n: number of variables
        coefficients (np.array): a numpy array of shape (n + 1, 1)
            - coefficients[0]: intercept
            - coefficients[1:]: remaining coefficients

    Returns:
        proba (float): the predicted probability for a data example.

    """

    # hint: you have to implement your z in a vectorized 
    # way aka using vector multiplications - it's different from what you have done above
    
    # hint: don't forget about adding an intercept to the train data!
    
    # YOUR CODE HERE
    train = np.append([1],train)
    z= np.matmul(train, coefficients)
    exp_z = np.exp(-z)
    proba = 1/(1+exp_z)
    
    #raise NotImplementedError()
    return proba

In [34]:
x = np.array([-1, -1])
coefficients = np.array([0 ,3.2, -1])

np.testing.assert_almost_equal(round(predict_proba(x, coefficients),3),0.1)

x_1 = np.array([-1, -1, 2, 0])
coefficients_1 = np.array([0 ,2, -1, 0.2, 0])

np.testing.assert_almost_equal(round(predict_proba(x_1, coefficients_1),3),0.354)

Expected output:

    Predicted probabilities for example with 2 variables:  0.0975
    
    Predicted probabilities for example with 3 variables:  0.3543

# Exercise 3: Compute the Maximum Log-Likelihood Cost Function

As you will implement stochastic gradient descent, you only have to do the following for each prediction, checking how much you will penalize each example according to the difference between the calculated probability and its true value: 

$$H_{\hat{p}}(y) =  - (y \log(\hat{p}) + (1-y) \log (1-\hat{p}))$$

In the next exercise you will loop through some examples stored in a array and calculate the cost function for the full dataset. Recall that the formula to generalize the cost function across several examples is: 

$$H_{\hat{p}}(y) = - \frac{1}{N}\sum_{i=1}^{N} \left [{ y_i \ \log(\hat{p}_i) + (1-y_i) \ \log (1-\hat{p}_i)} \right ]$$

You will basically simulate what stochastic gradient descent does without updating the coefficients - computing the log for each example, sum each log-loss and then averaging the result across the number of observations in the x dataset/array.

**Complete here:**

In [35]:
import math

def cost_function(x_values, coef, y):
    """ 
    Implementation of a function that returns the Maximum-Log-Likelihood loss
    
    Args:
        x_values (np.array): array with x training data of size (m, n) shape 
        where m is the number of observations and n the number of columns
        coef (float64): an array with the coefficients to apply of size (1, n+1)
        where n is the number of columns plus the intercept.
        y (float64): an array with integers with the real outcome per 
        example.
        
    Returns:
        loss (np.float): a float with the resulting log loss for the 
        entire data.

    """
    
    # A list of hints that you can follow:
    
    # you already computed a probability for an example
    # so you might be able to reuse the function
    coef_T = coef.reshape(-1,1)
    p_hat=[]
    #x_values = np.concatenate((np.ones(x_values.shape[0]).reshape(-1,1),x),axis=1)
    for row in range(x_values.shape[0]):
        p_prob = predict_proba(x_values[row], coef_T)
        p_hat.append(p_prob)
    p_hat = np.array(p_hat).reshape(1,-1)
    
    # Store number of examples that you have to loop through
    
    # Initialize loss
    
    # if you don't use the function from above to predict probas
    # don't forget to add the intercept to the X_array!
    
    # loop through every example
    
    # Calculate probability for each example
    # Compute log loss
    total_loss = -(np.dot(np.log(p_hat),y)+ np.dot(np.log(1-p_hat),(1-y)))/(y.shape[0])
    # Hint: maybe separating the log loss will help you
    # avoiding get confused inside all the parenthesis
    
    # Sum the computed loss for the example to the total log loss
    
    # Divide log loss by the number of examples (don't forget that the log loss
    # has to return a positive number!)    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return total_loss[0][0]

In [36]:
x = np.array([[-1, -1], [3, 0], [3, 2]])
coefficients = np.array([[0 ,2, -1]])
y = np.array([[1],[1],[0]])
np.testing.assert_almost_equal(round(cost_function(x, coefficients, y),3),1.778)

x_1 = np.array([[-1, -1], [3, 0], [3, 2], [1, 0]])
y_1 = np.array([[1],[1],[0],[1]])

np.testing.assert_almost_equal(round(cost_function(x_1, coefficients, y_1),3),1.365)


Expected output:
    
    Computed log loss for first training set:  1.77796243
    
    Computed log loss for second training set:  1.36520382

# Exercise 4: Compute a first pass on Stochastic Gradient Descent

Now that we know how to calculate probabilities and the cost function, let's do an interesting exercise - computing the derivatives and updating our coefficients. Here you will do a full pass a bunch of examples, computing the gradient descent for each time you see one of them.

In this exercise, you should compute a single iteration of the gradient descent! 

You will basically use stochastic gradient descent but you will have to update the coefficients after
you see a new example - so each time your algorithm knows that he saw something way off (for example, 
returning a low probability for an example with outcome = 1) he will have a way (the gradient) to 
change the coefficients so that he is able to minimize the cost function.

## Quick reminders:

Remember our formulas for the gradient:

$$\beta_{0(t+1)} = \beta_{0(t)} - learning\_rate \frac{\partial H_{\hat{p}}(y)}{\partial \beta_{0(t)}}$$

$$\beta_{t+1} = \beta_t - learning\_rate \frac{\partial H_{\hat{p}}(y)}{\partial \beta_t}$$

which can be simplified to

$$\beta_{0(t+1)} = \beta_{0(t)} + learning\_rate \left [(y - \hat{p}) \ \hat{p} \ (1 - \hat{p})\right]$$

$$\beta_{t+1} = \beta_t + learning\_rate \left [(y - \hat{p}) \ \hat{p} \ (1 - \hat{p}) \ x \right]$$

You will have to initialize the coefficients in some way. If you have a training set $X$, you can initialize them to zero, this way:
```python
coefficients = np.zeros(X.shape[1]+1)
```

where the $+1$ is adding the intercept.

Note: We are doing a stochastic gradient descent so don't forget to go observation by observation and updating the coefficients everytime!

**Complete here:**

In [37]:
def compute_coefficients(x_train, y_train, learning_rate = 0.1, verbose = False):
    """ 
    Implementation of a function that returns the a first iteration of 
    stochastic gradient descent.

    Args:
        x_train (np.array): a numpy array of shape (m, n)
            m: number of training observations
            n: number of variables
        y_train (np.array): a numpy array of shape (m,) with 
        the real value of the target.
        learning_rate (np.float64): a float

    Returns:
        coefficients (np.array): a numpy array of shape (n+1,)

    """
    
    # A list of hints that might help you:
    
    # Number of observations
    
    # initialize the coefficients array with zeros
    # hint: use np.zeros()
    m=x_train.shape[0]
    n=x_train.shape[1]
    b_old =  np.zeros((n+1))
    b_new = np.zeros((n+1))
    for j in range(m):
        p_hat = predict_proba(x_train[j], b_new)
        b_new[0] = b_old[0] + learning_rate*(y_train[j]-p_hat)*p_hat*(1-p_hat)
        for i in range(1,n):
                b_new[i] = b_old[i] + learning_rate*(y_train[j]-p_hat)*p_hat*(1-p_hat)*x_train[j][i]
        
    # run the stochastic gradient descent and update the coefficients after 
    # each observation    
    
    # compute the predicted probability - you can use a function we have done previously 
    
    # Update intercept
    
    # Update the rest of the coefficients by looping through each variable
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    coefficients = b_new
    return coefficients

In [38]:
x_train = np.array([[1,2,3], [2,5,9], [3,1,4], [8,2,9]])
y_train = np.array([0,1,0,1])
learning_rate = 0.1

np.testing.assert_almost_equal(round(compute_coefficients(x_train, y_train, learning_rate)[0],3),-0.001)

np.testing.assert_almost_equal(round(compute_coefficients(x_train, y_train, learning_rate)[1],3),0.064)

np.testing.assert_almost_equal(round(compute_coefficients(x_train, y_train, learning_rate)[2],3),0.056)

np.testing.assert_almost_equal(round(compute_coefficients(x_train, y_train, learning_rate)[3],3),0.139)

x_train_1 = np.array([[4,5,2,7], [2,5,7,2], [3,1,2,1], [8,2,9,5], [1,2,9,4]])
y_train_1 = np.array([0,1,0,1,1])

np.testing.assert_almost_equal(round(compute_coefficients(x_train_1, y_train_1, learning_rate).max(),3) ,0.198)
np.testing.assert_almost_equal(round(compute_coefficients(x_train_1, y_train_1, learning_rate).min(),3) ,0.006)
np.testing.assert_almost_equal(round(compute_coefficients(x_train_1, y_train_1, learning_rate).mean(),3) ,0.06)
np.testing.assert_almost_equal(round(compute_coefficients(x_train_1, y_train_1, learning_rate).var(),3) ,0.005)


AssertionError: 
Arrays are not almost equal to 7 decimals
 ACTUAL: 0.014
 DESIRED: -0.001

# Exercise 5: Normalize Data

To get this concept in your head, let's do a quick and easy function to normalize the data using a MaxMin approach. It is crucial that your variables are adjusted between $[0;1]$ (normalized) or standardized so that you can correctly analyse some logistic regression coefficients for your possible future employer.

You only have to implement this formula

$$ x_{normalized} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

Don't forget that the `axis` argument is critical when obtaining the maximum, minimum and mean values! As you want to obtain the maximum and minimum values of each individual feature, you have to specify `axis=0`. Thus, if you wanted to obtain the maximum values of each feature of data $X$, you would do the following:

```python
X_max = np.max(X, axis=0)
```

Not an assertable question but can you remember why it is important to normalize data for Logistic Regression?

**Complete here:**

In [39]:
a = np.arange(9).reshape(3,3)
a
a[:,0]

array([0, 3, 6])

In [40]:
def normalize_data(X):
    """ 
    Implementation of a function that normalizes your data variables
    
    Args:
        X (np.array): a numpy array of shape (m, n)
            m: number of observations
            n: number of variables

    Returns:
        normalized_X (np.array): a numpy array of shape (m, n)

    """
    # Compute the numerator first 
    # you can use np.min()
    min_features = np.zeros(X.shape[1])
    min_features =  np.min(X,axis = 0)
    
    # Compute the denominator
    # you can use np.max() and np.min()
    max_features = np.zeros(X.shape[1])
    max_features=np.max(X,axis = 0)
    normalized_X = np.zeros((X.shape[0],X.shape[1]))
    for j in range(X.shape[1]):
        normalized_X[:,j]=(X[:,j]-min_features[j])/(max_features[j]-min_features[j]) 
    # YOUR CODE HERE
    #raise NotImplementedError()
    return normalized_X

In [41]:
data = np.array([[7,7,3], [2,2,11], [9,5,2], [0,9,5], [10,1,3], [1,5,2]])
normalized_data = normalize_data(data)
print('Before normalization:')
print(data)
print('\n-------------------\n')
print('After normalization:')
print(normalized_data)

Before normalization:
[[ 7  7  3]
 [ 2  2 11]
 [ 9  5  2]
 [ 0  9  5]
 [10  1  3]
 [ 1  5  2]]

-------------------

After normalization:
[[0.7        0.75       0.11111111]
 [0.2        0.125      1.        ]
 [0.9        0.5        0.        ]
 [0.         1.         0.33333333]
 [1.         0.         0.11111111]
 [0.1        0.5        0.        ]]


Expected output:
    
    Before normalization:
    [[ 7  7  3]
     [ 2  2 11]
     [ 9  5  2]
     [ 0  9  5]
     [10  1  3]
     [ 1  5  2]]

    -------------------

After normalization:

    [[0.7        0.75       0.11111111]
     [0.2        0.125      1.        ]
     [0.9        0.5        0.        ]
     [0.         1.         0.33333333]
     [1.         0.         0.11111111]
     [0.1        0.5        0.        ]]

In [42]:
data = np.array([[2,2,11,1], [7,5,1,3], [9,5,2,6]])
normalized_data = normalize_data(data)
np.testing.assert_almost_equal(round(normalized_data.max(),3),1.0)
np.testing.assert_almost_equal(round(normalized_data.mean(),3),0.518)
np.testing.assert_almost_equal(round(normalized_data.var(),3),0.205)


data = np.array([[1,3,1,3], [9,5,3,1], [2,2,4,6]])
normalized_data = normalize_data(data)
np.testing.assert_almost_equal(round(normalized_data.mean(),3),0.460)
np.testing.assert_almost_equal(round(normalized_data.std(),3),0.427)

# Exercise 6: Training a Logistic Regression with Sklearn

In this exercise we will load the Titanic dataset and try to use the available numerical variables to predict the probability of a person surviving the titanic sinking.

Prepare to use your sklearn skills!

In [43]:
# We will load the dataset for you
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500

In this exercise you need to do the following: 
    - Select an array/Series with the target variable (Survived)
    - Select an array/dataframe with the X numeric variables (Pclass, Age, Siblings/Spouses Aboard, Parents/Children Aboard and Fare)
    - Scale all the X variables - normalize using Max / Min method.
    - Fit a logistic regression for maximum of 100 epochs and random state = 100.
    - Return an array of the predicted probas and return the coefficients
    
After this, feel free to explore your predictions! As a bonus why don't your construct a decision boundary using two variables eh? :-) 

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler


def train_model(dataset):
    '''
    Returns the predicted probas and coefficients 
    of a trained logistic regression on the Titanic Dataset.
    
    Args:
        dataset(pd.DataFrame): dataset to train on.
    
    Returns:
        probas (np.array): Array of floats with the probability 
        of surviving for each passenger
        coefficients (np.array): Returned coefficients of the 
        trained logistic regression.
    '''
    
    # leave this np.random seed here
    
    np.random.seed(100)
    X_train = dataset[['Pclass', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard','Fare']].to_numpy()
    y = dataset['Survived'].to_numpy()
    scaler = MinMaxScaler(feature_range=(0, 1))
# Fit your class
    scaler.fit(X_train)
# Transform your data
    X_train = scaler.transform(X_train)
    logit_clf = LogisticRegression(random_state=100,max_iter=100)
    logit_clf.fit(X_train, y)
    coef = logit_clf.coef_
    #intercept = logit_clf.intercept_
    #coef = np.concatenate((np.array([intercept]),coefficients.T))
    probas = logit_clf.predict_proba(X_train)
    probas = probas[:,1]
    
    # List of hints:
    
    # Use the Survived variable as y
    # Select the Numerical variables for X 
    # hint: use pandas .loc or indexing!    
    
    # Scale the X dataset - you can use a function we have already
    # constructed or resort to the sklearn implementation
    
    # Hint: for epochs look at the max_iter hyper param!
    # Fit logistic
    
    # Obtain probability of surviving
    

    # Obtain Coefficients from logistic regression
    # Hint: see the sklearn logistic regression documentation
    # if you do not know how to do this
    # No need to return the intercept, just the variable coefficients!
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return probas, coef
    

In [45]:
probas, coef = train_model(titanic)

# Testing Probas
max_probas = probas.max()
np.testing.assert_almost_equal(max_probas, 0.892, 2)
min_probas = probas.min()
np.testing.assert_almost_equal(min_probas, 0.066, 2)
mean_probas = probas.mean()
np.testing.assert_almost_equal(mean_probas, 0.386, 2)
std_probas = probas.std()
np.testing.assert_almost_equal(std_probas, 0.183, 2)
std_probas = probas.sum()
np.testing.assert_almost_equal(std_probas*0.001, 0.340, 2)
# Testing Coefs
max_coef = coef[0].max()
np.testing.assert_almost_equal(max_coef*0.1, 0.11, 1)
min_coef = coef[0].min()
np.testing.assert_almost_equal(min_coef*0.1, -0.25, 1)
mean_coef = coef[0].mean()
np.testing.assert_almost_equal(mean_coef*0.1, -0.07, 1)
std_coef = coef[0].std()
np.testing.assert_almost_equal(std_coef*0.1, 0.15, 1)
std_probas = coef[0].sum()
np.testing.assert_almost_equal(std_probas*0.1, -0.3, 1)